# Week 1 HW 1

In [55]:
import lucem_illud #pip install git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`
#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage
#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories


## Question 1

In [21]:
url = 'http://www.cnn.com/2018/01/09/politics/republicans-house-gop/index.html'
WebRequest = requests.get(url)
WebContent = bs4.BeautifulSoup(WebRequest.text,'html.parser')

In [83]:
# (1) find paragraph
Paragraphs = []
WebPar = WebContent.findAll('div', class_='zn-body__paragraph')

# (2) BUT there are reference numbers, remove reference numbers
for item in WebPar:
    Paragraphs.append(re.sub(r'\[\d+\]','', item.text))  # re.1
ParDF = pandas.DataFrame({'Paragraph':Paragraphs})
#print(ParDF)

# (3) Replace all "the United States" with "U.S."
replaceUS = []
for item in WebPar:
    replaceUS.append(re.sub(r'(the United States)', r'U.S.',item.text)) # re.2
ParDF['Replace "the United States" with "U.S."']=replaceUS
ParDF

# (4) Look for all paragrphs with "Democratic"
import string
Dem = ['1']*len(replaceUS)
for item in WebPar:
    for x in range(0,len(Dem)):
        if re.search(r'Democratic',item.text) is not None: # re.3
            Dem[x] = item.tex
ParDF['Democratic Paragraphs']=Dem
ParDF


,Paragraph,"Replace ""the United States"" with ""U.S.""",Democratic Paragraphs
0,Issa's retirement comes hot on the heels of hi...,Issa's retirement comes hot on the heels of hi...,None
1,"Royce, who sits atop the House Foreign Relatio...","Royce, who sits atop the House Foreign Relatio...",None
2,Issa and Royce are the 31st and 32nd Republica...,Issa and Royce are the 31st and 32nd Republica...,None
3,"There are, as always, mitigating circumstances...","There are, as always, mitigating circumstances...",None
4,"""There is going to be a GOP congressional wipe...","""There is going to be a GOP congressional wipe...",None
5,"And, Royce, like several other committee chair...","And, Royce, like several other committee chair...",None
6,But if you take a step back from any individua...,But if you take a step back from any individua...,None
7,Issa and Royce joins Rep. Ileana Ros-Lehtinen ...,Issa and Royce joins Rep. Ileana Ros-Lehtinen ...,None
8,"In each of those cases, the retiring members a...","In each of those cases, the retiring members a...",None
9,Why? Some of it has to do with the fact that t...,Why? Some of it has to do with the fact that t...,None


## Question 2

,Tagore Poem
0,﻿Where the mind is Without Fear \nWhere the mi...
1,"﻿Clouds And Waves\nMother, the folk who live u..."
2,﻿Closed Path\nI thought that my voyage had com...


## Question 3

In [110]:
# 1. for .txt file -- Tagore Poem
Tagore = ['Tagore1.txt','Tagore2.txt','Tagore3.txt']
TagorePoem =[]
for x in range(0,3):
    with open(Tagore[x], encoding='utf-8') as f:
        TagorePoem.append(f.read())
ALL = pandas.DataFrame({'Tagore Poem': TagorePoem})

In [117]:
# 2. for .PDF file -- Love Poem
information_extraction_pdf = ['https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf',
                              'https://www.federalreserve.gov/econresdata/feds/2015/files/2015083pap.pdf',
                              'https://www.kent.ac.uk/economics/documents/research/papers/2009/0925.pdf']

EconomicPaper = []
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString
for x in range(0,3):
    infoExtractionRequest = requests.get(information_extraction_pdf[x], stream=True)
    infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)
    EconomicPaper.append(readPDF(infoExtractionBytes)[:300])
ALL['Economic Paper'] = EconomicPaper

,Tagore Poem,Economic Paper
0,﻿Where the mind is Without Fear \nWhere the mi...,Speech and Language Processing. Daniel Jurafsk...
1,"﻿Clouds And Waves\nMother, the folk who live u...",Finance and Economics Discussion Series\n\nDiv...
2,﻿Closed Path\nI thought that my voyage had com...,\n\n \n\nUniversity of Kent \n\nSchool of Eco...


In [ ]:
# 3. for .doc file
